In [29]:
# multivariate output 1d cnn example
from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from sklearn.model_selection import train_test_split
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from numpy import loadtxt
lines = loadtxt("trial0_revised11.txt", delimiter=",", unpack=False, usecols=(4,7,10,13,16,19,22,25,28,31))
print(lines[0][0])


0.160804


In [30]:
len(lines)

24003

In [31]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	print(len(sequences))
	for i in range(len(sequences)):
		seq_x, seq_y = sequences[i, :7], sequences[i, 7:]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


In [32]:

# horizontally stack columns
#dataset = hstack((in_seq1, in_seq2, out_seq))
#print(dataset)
# choose a number of time steps
n_steps = 3


In [33]:
# convert into input/output
X, y = split_sequences(lines, n_steps)
#X=X[None,:,:]
print(X.shape)
print(y.shape)

24003
(24003, 7)
(24003, 3)


In [34]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[-1]
print('shape='+str(n_features))

shape=7


In [35]:
visible = Input(shape=(7, 1))
cnn = Conv1D(filters=64, kernel_size=2, activation='relu')(visible)
cnn = MaxPooling1D(pool_size=2)(cnn)
cnn = Flatten()(cnn)
cnn = Dense(50, activation='relu')(cnn)
# define output 1
output1 = Dense(1)(cnn)
# define output 2
output2 = Dense(1)(cnn)
# define output 3
output3 = Dense(1)(cnn)


In [36]:
# tie together
model = Model(inputs=visible, outputs=[output1, output2, output3])
model.compile(optimizer='adam', loss='mse')
print(model.summary())


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 7, 1)]       0           []                               
                                                                                                  
 conv1d_2 (Conv1D)              (None, 6, 64)        192         ['input_3[0][0]']                
                                                                                                  
 max_pooling1d_2 (MaxPooling1D)  (None, 3, 64)       0           ['conv1d_2[0][0]']               
                                                                                                  
 flatten_2 (Flatten)            (None, 192)          0           ['max_pooling1d_2[0][0]']        
                                                                                            

In [37]:

print(y.shape)
y1 = y[:, 0].reshape((y.shape[0], 1))
y2 = y[:, 1].reshape((y.shape[0], 1))
y3 = y[:, 2].reshape((y.shape[0], 1))
print(X.shape)
print(y1.shape)

(24003, 3)
(24003, 7)
(24003, 1)


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75) # [y1,y2,y3]
# fit model
print('fitting')
print(y_train.shape)
ytr1 = y_train[:, 0].reshape((y_train.shape[0], 1))
ytr2 = y_train[:, 1].reshape((y_train.shape[0], 1))
ytr3 = y_train[:, 2].reshape((y_train.shape[0], 1))
yte1 = y_test[:, 0].reshape((y_test.shape[0], 1))
yte2 = y_test[:, 1].reshape((y_test.shape[0], 1))
yte3 = y_test[:, 2].reshape((y_test.shape[0], 1))

model.fit(X_train, [ytr1,ytr2,ytr3], epochs=20,batch_size=64)

fitting
(18002, 3)
Epoch 1/20
282/282 [==============================] - 1s 3ms/step - loss: 1.2173e-04 - dense_9_loss: 3.5435e-05 - dense_10_loss: 3.8262e-05 - dense_11_loss: 4.8030e-05
Epoch 2/20
282/282 [==============================] - 1s 3ms/step - loss: 1.2218e-04 - dense_9_loss: 3.5006e-05 - dense_10_loss: 3.9027e-05 - dense_11_loss: 4.8144e-05
Epoch 3/20
282/282 [==============================] - 1s 4ms/step - loss: 1.2415e-04 - dense_9_loss: 3.6971e-05 - dense_10_loss: 3.9114e-05 - dense_11_loss: 4.8067e-05
Epoch 4/20
282/282 [==============================] - 1s 3ms/step - loss: 1.2483e-04 - dense_9_loss: 3.7030e-05 - dense_10_loss: 3.9263e-05 - dense_11_loss: 4.8532e-05
Epoch 5/20
282/282 [==============================] - 1s 4ms/step - loss: 1.2508e-04 - dense_9_loss: 3.7062e-05 - dense_10_loss: 3.9528e-05 - dense_11_loss: 4.8486e-05
Epoch 6/20
282/282 [==============================] - 1s 3ms/step - loss: 1.2523e-04 - dense_9_loss: 3.7140e-05 - dense_10_loss: 3.9726e-05 -

In [44]:

score = model.evaluate(X_test, [yte1,yte2,yte3], verbose = 0) 

print("test loss, test acc:", score)

test loss, test acc: [0.0001368911616737023, 4.616444857674651e-05, 4.008636824437417e-05, 5.0640290282899514e-05]


In [40]:
lines2 = loadtxt("trial3_revised11.txt", delimiter=",", unpack=False, usecols=(4,7,10,13,16,19,22,25,28,31))

In [41]:
seq_x2, seq_y2 = lines2[100, :7], lines2[100, 7:]
x2=list()
x2.append(seq_x2)
x2=array(x2)
print(x2.shape)

yhat = model.predict(x2, verbose=0)
print(yhat)
print(seq_y2)

(1, 7)
[array([[-0.3015609]], dtype=float32), array([[-0.228706]], dtype=float32), array([[-0.2573509]], dtype=float32)]
[ 0.151382 -0.02439  -0.064336]
